In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
import psycopg2
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
conn = psycopg2.connect("dbname=myduka user=postgres password=Candidcoco@20")

In [4]:
df = pd.read_sql_query("SELECT sales.id as sales_id, product_name, buying_price, selling_price, sales.quantity as sales_quantity, created_at as sales_created_at FROM products,sales WHERE products.id = sales.pid", conn) 

In [5]:
df.set_index('sales_id', inplace=True,verify_integrity=False)

In [6]:
df['profit'] = df['selling_price'] - df['buying_price']

In [7]:
df['new_selling_price'] = df['buying_price'] * 0.25

In [8]:
df['new_selling_price'] = df['new_selling_price'] * 10

In [9]:
df['new_profit'] = (df['new_selling_price'] - df['buying_price']) * df['sales_quantity']

In [10]:
df.drop(columns=['selling_price', 'profit'], inplace=True)

In [11]:
column_names = {'new_selling_price': 'selling_price', 'new_profit': 'profit'}
df.rename(columns=column_names, inplace=True)

In [12]:
df.head(5)

,product_name,buying_price,sales_quantity,sales_created_at,selling_price,profit
sales_id,,,,,,
1,Vinegar - Tarragon,65,23,2022-11-13,162.5,2242.5
2,Cloves - Whole,33,12,2022-08-31,82.5,594.0
3,"Pail - 15l White, With Handle",47,92,2022-12-29,117.5,6486.0
4,"Chocolate - Pistoles, Lactee, Milk",75,17,2023-04-08,187.5,1912.5
5,Cassis,16,100,2023-05-25,40.0,2400.0


In [13]:
X = df[['selling_price','sales_quantity']]
y = df['profit'] 

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 104, test_size=0.25, shuffle = True)

In [15]:
regr = LinearRegression()
# Train the model using the training sets
regr.fit(X_train, y_train)

LinearRegression()

In [16]:
# Make predictions using the testing set
y_test_pred = regr.predict(X_test)
y_train_pred = regr.predict(X_train)

In [17]:
print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train: 957852.919, test: 1824659.372
R^2 train: 0.899, test: 0.840
